# Before 9:30

In [65]:
import pandas as pd


instruments_df = pd.read_csv('DataSets/example-set/input_instruments.csv')
orders_df =  pd.read_csv('DataSets/example-set/input_orders.csv')
clients_df = pd.read_csv('DataSets/example-set/input_clients.csv')

clients_df.set_index('ClientID', inplace=True)
orders_df['Rating'] = orders_df['Client'].map(clients_df['Rating'])
orders_df['Currencies'] = orders_df['Client'].map(clients_df['Currencies'])
orders_df['PositionCheck'] = orders_df['Client'].map(clients_df['PositionCheck'])

instruments_df.set_index('InstrumentID', inplace=True)
orders_df['LotSize'] = orders_df['Instrument'].map(instruments_df['LotSize'])

clients_df.reset_index(inplace=True)
instruments_df.reset_index(inplace=True)

orders_df



,Time,OrderID,Instrument,Quantity,Client,Price,Side,Rating,Currencies,PositionCheck,LotSize
0,9:00:01,A1,SIA,1500,A,Market,Buy,1,"USD,SGD",Y,100
1,9:02:00,B1,SIA,4500,B,32.1,Sell,2,"USD,SGD,JPY",N,100
2,9:05:00,C1,SIA,100,C,32,Buy,3,SGD,Y,100
3,9:10:00,D1,SIA,300,D,Market,Sell,4,USD,Y,100
4,9:29:01,B2,SIA,5,B,32.1,Sell,2,"USD,SGD,JPY",N,100
5,9:29:02,E1,SIA,1000,E,32,Sell,5,SGD,N,100
6,9:29:03,A2,SIA,800,A,31.9,Buy,1,"USD,SGD",Y,100
7,9:30:01,C2,SIA,100,C,Market,Sell,3,SGD,Y,100
8,9:40:00,B3,SIA,500,B,32.2,Sell,2,"USD,SGD,JPY",N,100
9,10:50:00,C3,SIA,4200,C,32.2,Buy,3,SGD,Y,100


In [66]:
def validate_instruments(instruments_df, orders_df):
    instrument_ids_set = set(instruments_df['InstrumentID'])
    orders_df['Instrument_Valid'] = orders_df['Instrument'].isin(instrument_ids_set)

    orders_df['RejectionReason'] = ''

    orders_df.loc[orders_df['Instrument_Valid'] == False, 'RejectionReason'] = 'REJECTED - INSTRUMENT NOT FOUND'
    return orders_df

validate_instruments(instruments_df, orders_df)
orders_df

,Time,OrderID,Instrument,Quantity,Client,Price,Side,Rating,Currencies,PositionCheck,LotSize,Instrument_Valid,RejectionReason
0,9:00:01,A1,SIA,1500,A,Market,Buy,1,"USD,SGD",Y,100,True,
1,9:02:00,B1,SIA,4500,B,32.1,Sell,2,"USD,SGD,JPY",N,100,True,
2,9:05:00,C1,SIA,100,C,32,Buy,3,SGD,Y,100,True,
3,9:10:00,D1,SIA,300,D,Market,Sell,4,USD,Y,100,True,
4,9:29:01,B2,SIA,5,B,32.1,Sell,2,"USD,SGD,JPY",N,100,True,
5,9:29:02,E1,SIA,1000,E,32,Sell,5,SGD,N,100,True,
6,9:29:03,A2,SIA,800,A,31.9,Buy,1,"USD,SGD",Y,100,True,
7,9:30:01,C2,SIA,100,C,Market,Sell,3,SGD,Y,100,True,
8,9:40:00,B3,SIA,500,B,32.2,Sell,2,"USD,SGD,JPY",N,100,True,
9,10:50:00,C3,SIA,4200,C,32.2,Buy,3,SGD,Y,100,True,


In [67]:
def validate_currency(instruments_df, orders_df):
    currency_set = set(instruments_df['Currency'])

    for index, order in orders_df.iterrows():
        if not order['RejectionReason']:
            client_currencies = set(order['Currencies'].split(','))
            is_valid = any(currency in currency_set for currency in client_currencies)
            orders_df.at[index, 'Currency_Valid'] = is_valid

    orders_df.loc[orders_df['Currency_Valid'] == False, 'RejectionReason'] = 'REJECTED - MISMATCH CURRENCY'

    return orders_df

validate_currency(instruments_df, orders_df)


,Time,OrderID,Instrument,Quantity,Client,Price,Side,Rating,Currencies,PositionCheck,LotSize,Instrument_Valid,RejectionReason,Currency_Valid
0,9:00:01,A1,SIA,1500,A,Market,Buy,1,"USD,SGD",Y,100,True,,True
1,9:02:00,B1,SIA,4500,B,32.1,Sell,2,"USD,SGD,JPY",N,100,True,,True
2,9:05:00,C1,SIA,100,C,32,Buy,3,SGD,Y,100,True,,True
3,9:10:00,D1,SIA,300,D,Market,Sell,4,USD,Y,100,True,REJECTED - MISMATCH CURRENCY,False
4,9:29:01,B2,SIA,5,B,32.1,Sell,2,"USD,SGD,JPY",N,100,True,,True
5,9:29:02,E1,SIA,1000,E,32,Sell,5,SGD,N,100,True,,True
6,9:29:03,A2,SIA,800,A,31.9,Buy,1,"USD,SGD",Y,100,True,,True
7,9:30:01,C2,SIA,100,C,Market,Sell,3,SGD,Y,100,True,,True
8,9:40:00,B3,SIA,500,B,32.2,Sell,2,"USD,SGD,JPY",N,100,True,,True
9,10:50:00,C3,SIA,4200,C,32.2,Buy,3,SGD,Y,100,True,,True


In [68]:
def validate_lotsize(orders_df):
  for i in range(0, len(orders_df)):
    if not orders_df.loc[i, 'RejectionReason']:
      if (orders_df.loc[i, 'Quantity'] % orders_df.loc[i, 'LotSize']) == 0:
        orders_df.loc[i, 'Lotsize_valid'] = True
      else:
        orders_df.loc[i, 'Lotsize_valid'] = False
        orders_df.loc[i, 'RejectionReason'] = "REJECTED - INVLALID LOT SIZE"

  return orders_df

validate_lotsize(orders_df)

,Time,OrderID,Instrument,Quantity,Client,Price,Side,Rating,Currencies,PositionCheck,LotSize,Instrument_Valid,RejectionReason,Currency_Valid,Lotsize_valid
0,9:00:01,A1,SIA,1500,A,Market,Buy,1,"USD,SGD",Y,100,True,,True,True
1,9:02:00,B1,SIA,4500,B,32.1,Sell,2,"USD,SGD,JPY",N,100,True,,True,True
2,9:05:00,C1,SIA,100,C,32,Buy,3,SGD,Y,100,True,,True,True
3,9:10:00,D1,SIA,300,D,Market,Sell,4,USD,Y,100,True,REJECTED - MISMATCH CURRENCY,False,NaN
4,9:29:01,B2,SIA,5,B,32.1,Sell,2,"USD,SGD,JPY",N,100,True,REJECTED - INVLALID LOT SIZE,True,False
5,9:29:02,E1,SIA,1000,E,32,Sell,5,SGD,N,100,True,,True,True
6,9:29:03,A2,SIA,800,A,31.9,Buy,1,"USD,SGD",Y,100,True,,True,True
7,9:30:01,C2,SIA,100,C,Market,Sell,3,SGD,Y,100,True,,True,True
8,9:40:00,B3,SIA,500,B,32.2,Sell,2,"USD,SGD,JPY",N,100,True,,True,True
9,10:50:00,C3,SIA,4200,C,32.2,Buy,3,SGD,Y,100,True,,True,True


In [69]:
def validate_position_before_930(orders_df):
    for index, order in orders_df.iterrows():
        if not order['RejectionReason']:
            if order['PositionCheck'] == 'Y' and order['Side'] == 'Sell':
                orders_df.at[index, 'Position_Valid'] = False
            else:
                orders_df.at[index, 'Position_Valid'] = True

    orders_df.loc[orders_df['Position_Valid'] == False, 'RejectionReason'] = 'REJECTED - POSITION CHECK FAILED'

    return orders_df

validate_position_before_930(orders_df)

,Time,OrderID,Instrument,Quantity,Client,Price,Side,Rating,Currencies,PositionCheck,LotSize,Instrument_Valid,RejectionReason,Currency_Valid,Lotsize_valid,Position_Valid
0,9:00:01,A1,SIA,1500,A,Market,Buy,1,"USD,SGD",Y,100,True,,True,True,True
1,9:02:00,B1,SIA,4500,B,32.1,Sell,2,"USD,SGD,JPY",N,100,True,,True,True,True
2,9:05:00,C1,SIA,100,C,32,Buy,3,SGD,Y,100,True,,True,True,True
3,9:10:00,D1,SIA,300,D,Market,Sell,4,USD,Y,100,True,REJECTED - MISMATCH CURRENCY,False,NaN,NaN
4,9:29:01,B2,SIA,5,B,32.1,Sell,2,"USD,SGD,JPY",N,100,True,REJECTED - INVLALID LOT SIZE,True,False,NaN
5,9:29:02,E1,SIA,1000,E,32,Sell,5,SGD,N,100,True,,True,True,True
6,9:29:03,A2,SIA,800,A,31.9,Buy,1,"USD,SGD",Y,100,True,,True,True,True
7,9:30:01,C2,SIA,100,C,Market,Sell,3,SGD,Y,100,True,REJECTED - POSITION CHECK FAILED,True,True,False
8,9:40:00,B3,SIA,500,B,32.2,Sell,2,"USD,SGD,JPY",N,100,True,,True,True,True
9,10:50:00,C3,SIA,4200,C,32.2,Buy,3,SGD,Y,100,True,,True,True,True


In [70]:
orders_df.to_csv('DataSets/intermediate-set/orders_df_after_validation.csv', index=False)

...........

# New Trade

In [71]:
data = {
    'Time': ['09:30:01'],
    'Client': ['C'],
    'Instrument': ['SIA'],
    'Side': ['Sell'],
    'Price': ['Market'],
    'Quantity': [100],
    'OrderID': ['C2']
}

new_trade = pd.DataFrame(data)
prev_orders = pd.read_csv('DataSets/intermediate-set/previous-orders.csv')

new_trade


,Time,Client,Instrument,Side,Price,Quantity,OrderID
0,09:30:01,C,SIA,Sell,Market,100,C2


In [72]:
validate_instruments(instruments_df, new_trade)


,Time,Client,Instrument,Side,Price,Quantity,OrderID,Instrument_Valid,RejectionReason
0,09:30:01,C,SIA,Sell,Market,100,C2,True,


In [73]:
new_trade_currency = clients_df[clients_df['ClientID'] == new_trade['Client'].iloc[0]]['Currencies'].iloc[0]
new_trade['Currencies'] = new_trade_currency
new_trade

,Time,Client,Instrument,Side,Price,Quantity,OrderID,Instrument_Valid,RejectionReason,Currencies
0,09:30:01,C,SIA,Sell,Market,100,C2,True,,SGD


In [74]:
validate_currency(instruments_df, new_trade)

,Time,Client,Instrument,Side,Price,Quantity,OrderID,Instrument_Valid,RejectionReason,Currencies,Currency_Valid
0,09:30:01,C,SIA,Sell,Market,100,C2,True,,SGD,True


In [75]:
new_trade_lotsize = instruments_df[instruments_df['InstrumentID'] == new_trade['Instrument'].iloc[0]]['LotSize'].iloc[0]
new_trade['LotSize'] = new_trade_lotsize
new_trade

,Time,Client,Instrument,Side,Price,Quantity,OrderID,Instrument_Valid,RejectionReason,Currencies,Currency_Valid,LotSize
0,09:30:01,C,SIA,Sell,Market,100,C2,True,,SGD,True,100


In [76]:
validate_lotsize(new_trade)

,Time,Client,Instrument,Side,Price,Quantity,OrderID,Instrument_Valid,RejectionReason,Currencies,Currency_Valid,LotSize,Lotsize_valid
0,09:30:01,C,SIA,Sell,Market,100,C2,True,,SGD,True,100,True


In [77]:
import pandas as pd


def validate_position_continuous(new_trade, previous_orders):
    starting_letters = set(order[0] for order in previous_orders['SellOrder#'])
    if not any(new_trade['Client'].iloc[0].startswith(letter) for letter in starting_letters):
        new_trade.loc[0, 'Lotsize_valid'] = False
        new_trade.loc[0, 'RejectionReason'] = 'REJECTED - POSITION CHECK FAILED'
            
    return new_trade

validate_position_continuous(new_trade, prev_orders)


,Time,Client,Instrument,Side,Price,Quantity,OrderID,Instrument_Valid,RejectionReason,Currencies,Currency_Valid,LotSize,Lotsize_valid
0,09:30:01,C,SIA,Sell,Market,100,C2,True,REJECTED - POSITION CHECK FAILED,SGD,True,100,False


In [78]:
rejected = pd.DataFrame(columns=['OrderID', 'RejectionReason']) 
if new_trade['RejectionReason'].iloc[0] != '':
    rejected = rejected.append({'OrderID': new_trade['OrderID'].iloc[0], 'RejectionReason': new_trade['RejectionReason'].iloc[0]}, ignore_index=True)
rejected


/var/folders/m_/kb9mf0b90hb2fvtfccyd41fr0000gn/T/ipykernel_86480/4274313354.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rejected = rejected.append({'OrderID': new_trade['OrderID'].iloc[0], 'RejectionReason': new_trade['RejectionReason'].iloc[0]}, ignore_index=True)


,OrderID,RejectionReason
0,C2,REJECTED - POSITION CHECK FAILED
